In [1]:
import sys
sys.path.append("..")

In [2]:
import pickle

In [3]:
with open("../data/processed/players.p", "rb") as file_pointer:
    players = pickle.load(file_pointer)
    
with open("../data/processed/formations.p", "rb") as file_pointer:
    formations = pickle.load(file_pointer)

In [15]:
players = players[(players["price"] > 0) & (players["metal"] == "gold")]

KeyError: 'metal'

In [5]:
import numpy as np

In [6]:
from fut_squad_evolver.fut_elements.genotype import Squad
from fut_squad_evolver.fut_elements.initializer import SquadInitializer
from fut_squad_evolver.fut_elements.crossover import SquadCrossover
from fut_squad_evolver.fut_elements.mutator import SquadMutator
from fut_squad_evolver.nsga2.evaluator import Evaluator
from fut_squad_evolver.nsga2.selector import TournamentSelector

In [7]:
formation_442 = formations["442"]

In [8]:
initializer = SquadInitializer(1000, formation_442, players, 2)

In [9]:
greater_is_better_dict = {"price": False, "overall": True, "chemistry": True}

In [10]:
evaluator = Evaluator(non_dominated_sorter_kwargs={"greater_is_better_dict": greater_is_better_dict}, crowding_distance_sorter_kwargs={"greater_is_better_dict": greater_is_better_dict})
selector = TournamentSelector(1000)

In [11]:
crossover = SquadCrossover(0.2)
mutator = SquadMutator(players, 2, 0.75, 0.4, 0.3, 0.3)

In [12]:
population = initializer.initialize()
for individual in population.values():
    individual.evaluate()
_ = evaluator.evaluate(population)
for i in range(2000):
    _ = evaluator.evaluate(population)
    print(np.min([phenotype["price"] for phenotype in [individual.phenotype for individual in population.values() if individual.fitness["pareto_front"] == 0]]))
    print(np.max([phenotype["overall"] for phenotype in [individual.phenotype for individual in population.values() if individual.fitness["pareto_front"] == 0]]))
    print(np.max([phenotype["chemistry"] for phenotype in [individual.phenotype for individual in population.values() if individual.fitness["pareto_front"] == 0]]))
    print("")
    champions = selector.populate(population)
    new_population = {}
    for parent_a, parent_b in zip(champions[::2], champions[1::2]):
        child_a, child_b = crossover.crossover(parent_a, parent_b)
        new_population[child_a.individual_id] = child_a
        new_population[child_b.individual_id] = child_b
    for individual in new_population.values():
        mutator.mutate(individual)
    for individual in new_population.values():
        individual.evaluate()
    population = new_population

3250
75.36363636363636
63

3200
75.63636363636364
60

3200
76.27272727272727
60

2650
76.9090909090909
61

2850
76.81818181818181
64

2900
77.54545454545455
64

2850
77.54545454545455
64

2850
77.36363636363636
64

2850
77.54545454545455
73

2700
77.45454545454545
64

2700
78.54545454545455
65

2700
78.54545454545455
64

2700
78.54545454545455
67

2450
80.18181818181819
67

2450
78.54545454545455
74

2450
78.81818181818181
74

2450
80.45454545454545
80

2450
79.36363636363636
74

2450
79.0
74

2400
79.9090909090909
74

2400
79.0909090909091
71

2400
79.45454545454545
72

2400
79.45454545454545
77

2400
79.45454545454545
72

2400
80.72727272727273
80

2450
80.72727272727273
80

2500
80.72727272727273
75

2500
80.72727272727273
75

2500
80.72727272727273
81

2500
80.72727272727273
75

2500
80.72727272727273
75

2750
80.72727272727273
75

2500
80.72727272727273
75

2500
80.72727272727273
75

2700
80.72727272727273
75

2700
80.72727272727273
78

2700
80.81818181818181
78

2700
80.818181818

KeyboardInterrupt: 

In [ ]:
_ = evaluator.evaluate(population)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame([individual.phenotype for individual in population.values() if individual.fitness["pareto_front"] == 0 \
              and individual.phenotype["overall"] > 75 \
              and individual.phenotype["chemistry"] > 80]).sort_values("price").head(25)

In [ ]:
[individual for individual in population.values() if individual.individual_id == 3165878][0].genotype